# Tutorial 06, case 8: Navier-Stokes problem with Neumann control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} |v - v_d|^2 dx + \frac{\alpha}{2} \int_{\Gamma_2} |u|^2 ds$$
s.t.
$$\begin{cases}
- \nu \Delta v + v \cdot \nabla v + \nabla p = f       & \text{in } \Omega\\
                                \text{div} v = 0       & \text{in } \Omega\\
                                           v = 0       & \text{on } \Gamma_1\\
                       pn - \nu \partial_n v = u       & \text{on } \Gamma_2\\
                                           v = 0       & \text{on } \Gamma_3\\
                                           v = 0       & \text{on } \Gamma_4
\end{cases}$$

where
$$\begin{align*}
& \Omega                      & \text{unit square}\\
& \Gamma_1                    & \text{bottom boundary of the square}\\
& \Gamma_2                    & \text{left boundary of the square}\\
& \Gamma_3                    & \text{top boundary of the square}\\
& \Gamma_4                    & \text{right boundary of the square}\\
& u \in [L^2(\Gamma_2)]^2     & \text{control variable}\\
& v \in [H^1(\Omega)]^2       & \text{state velocity variable}\\
& p \in L^2(\Omega)           & \text{state pressure variable}\\
& \alpha > 0                  & \text{penalization parameter}\\
& v_d                         & \text{desired state}\\
& \nu                         & \text{kinematic viscosity}\\
& f                           & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
import numpy as np
from petsc4py import PETSc
import sympy
from ufl import derivative, div, grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, MPI, VectorFunctionSpace
from dolfinx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block, BlockVecSubVectorWrapper,
                         create_vector_block, create_matrix_block, DofMapRestriction, locate_dofs_topological)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Mesh

In [ ]:
with XDMFFile(MPI.comm_world, "data/square.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.create_connectivity_all()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    boundaries = infile.read_meshtags(mesh, name="boundaries")
boundaries_134 = boundaries.indices[np.isin(boundaries.values, (1, 3, 4))]
boundaries_2 = boundaries.indices[boundaries.values == 2]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Function spaces

In [ ]:
Y_velocity = VectorFunctionSpace(mesh, ("Lagrange", 2))
Y_pressure = FunctionSpace(mesh, ("Lagrange", 1))
U = VectorFunctionSpace(mesh, ("Lagrange", 2))
Q_velocity = Y_velocity.clone()
Q_pressure = Y_pressure.clone()

### Restrictions

In [ ]:
dofs_Y_velocity = np.arange(0, Y_velocity.dofmap.index_map.block_size * (
    Y_velocity.dofmap.index_map.size_local + Y_velocity.dofmap.index_map.num_ghosts))
dofs_Y_pressure = np.arange(0, Y_pressure.dofmap.index_map.block_size * (
    Y_pressure.dofmap.index_map.size_local + Y_pressure.dofmap.index_map.num_ghosts))
dofs_U = locate_dofs_topological(U, boundaries.dim, boundaries_2)
dofs_Q_velocity = dofs_Y_velocity
dofs_Q_pressure = dofs_Y_pressure
restriction_Y_velocity = DofMapRestriction(Y_velocity.dofmap, dofs_Y_velocity)
restriction_Y_pressure = DofMapRestriction(Y_pressure.dofmap, dofs_Y_pressure)
restriction_U = DofMapRestriction(U.dofmap, dofs_U)
restriction_Q_velocity = DofMapRestriction(Q_velocity.dofmap, dofs_Q_velocity)
restriction_Q_pressure = DofMapRestriction(Q_pressure.dofmap, dofs_Q_pressure)
restriction = [restriction_Y_velocity, restriction_Y_pressure, restriction_U,
               restriction_Q_velocity, restriction_Q_pressure]

### Trial and test functions

In [ ]:
(dv, dp) = (TrialFunction(Y_velocity), TrialFunction(Y_pressure))
(w, q) = (TestFunction(Y_velocity), TestFunction(Y_pressure))
du = TrialFunction(U)
r = TestFunction(U)
(dz, db) = (TrialFunction(Q_velocity), TrialFunction(Q_pressure))
(s, d) = (TestFunction(Q_velocity), TestFunction(Q_pressure))

### Solution

In [ ]:
(v, p) = (Function(Y_velocity), Function(Y_pressure))
u = Function(U)
(z, b) = (Function(Q_velocity), Function(Q_pressure))

 ### Problem data

In [ ]:
alpha = 1.e-5
x, y = sympy.symbols("x[0], x[1]")
psi_d = 10 * (1 - sympy.cos(0.8 * np.pi * x)) * (1 - sympy.cos(0.8 * np.pi * y)) * (1 - x)**2 * (1 - y)**2
v_d_x = sympy.lambdify([x, y], psi_d.diff(y, 1))
v_d_y = sympy.lambdify([x, y], - psi_d.diff(x, 1))
v_d = Function(Y_velocity)
v_d.interpolate(lambda x: np.stack((v_d_x(x[0], x[1]), v_d_y(x[0], x[1])), axis=0))
nu = 0.1
ff = Constant(mesh, (0., 0.))
bc0 = Function(Y_velocity)

### Optimality conditions

In [ ]:
F = [nu * inner(grad(z), grad(w)) * dx + inner(grad(w) * v, z) * dx
     + inner(grad(v) * w, z) * dx - b * div(w) * dx + inner(v - v_d, w) * dx,
     - q * div(z) * dx,
     alpha * inner(u, r) * ds(2) - inner(z, r) * ds(2),
     nu * inner(grad(v), grad(s)) * dx + inner(grad(v) * v, s) * dx - p * div(s) * dx
     - inner(ff, s) * dx - inner(u, s) * ds(2),
     - d * div(v) * dx]
dF = [[derivative(F_i, u_j, du_j) for (u_j, du_j) in zip((v, p, u, z, b), (dv, dp, du, dz, db))] for F_i in F]
dF[3][3] = Constant(mesh, 0.) * inner(dz, s) * (ds(1) + ds(3) + ds(4))
bdofs_Y_velocity_134 = locate_dofs_topological((Y_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_134)
bdofs_Q_velocity_134 = locate_dofs_topological((Q_velocity, Y_velocity), mesh.topology.dim - 1, boundaries_134)
bc = [DirichletBC(bc0, bdofs_Y_velocity_134, Y_velocity),
      DirichletBC(bc0, bdofs_Q_velocity_134, Q_velocity)]

### Cost functional

In [ ]:
J = 0.5 * inner(v - v_d, v - v_d) * dx + 0.5 * alpha * inner(u, u) * ds(2)

### Class for interfacing with SNES

In [ ]:
class NonlinearBlockProblem(object):
    def __init__(self, F, dF, solutions, bcs, restriction=None):
        self._F = F
        self._dF = dF
        self._obj_vec = create_vector_block(F, restriction)
        self._solutions = solutions
        self._bcs = bcs
        self._restriction = restriction

    def update_solutions(self, x):
        x.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        with BlockVecSubVectorWrapper(x, [c.function_space.dofmap for c in self._solutions],
                                      self._restriction) as x_wrapper:
            for x_wrapper_local, component in zip(x_wrapper, self._solutions):
                with component.vector.localForm() as component_local:
                    component_local[:] = x_wrapper_local

    def obj(self, snes, x):
        self.F(snes, x, self._obj_vec)
        return self._obj_vec.norm()

    def F(self, snes, x, F_vec):
        self.update_solutions(x)
        with F_vec.localForm() as F_vec_local:
            F_vec_local.set(0.0)
        assemble_vector_block(F_vec, self._F, self._dF, self._bcs, x0=x, scale=-1.0,
                              restriction=self._restriction, restriction_x0=self._restriction)

    def dF(self, snes, x, dF_mat, _):
        dF_mat.zeroEntries()
        if self._restriction is None:
            restriction = None
        else:
            restriction = (self._restriction, self._restriction)
        assemble_matrix_block(dF_mat, self._dF, self._bcs, diagonal=1.0,
                              restriction=restriction)
        dF_mat.assemble()

### Uncontrolled functional value

In [ ]:
# Create problem by extracting state forms from the optimality conditions
F_state = [replace(F[i], {s: w, d: q, u: Constant(mesh, (0, 0))}) for i in (3, 4)]
dF_state = [[derivative(Fs_i, u_j, du_j) for (u_j, du_j) in zip((v, p), (dv, dp))] for Fs_i in F_state]
dF_state[1][1] = Constant(mesh, 0) * dp * q * dx
bc_state = [bc[0]]
problem_state = NonlinearBlockProblem(F_state, dF_state, (v, p), bc_state)
F_vec_state = create_vector_block(F_state)
dF_mat_state = create_matrix_block(dF_state)

In [ ]:
# Solve
vp = create_vector_block([F[j] for j in (0, 1)])
snes = PETSc.SNES().create(mesh.mpi_comm())
snes.setTolerances(max_it=20)
snes.getKSP().setType("preonly")
snes.getKSP().getPC().setType("lu")
snes.getKSP().getPC().setFactorSolverType("mumps")
snes.setObjective(problem_state.obj)
snes.setFunction(problem_state.F, F_vec_state)
snes.setJacobian(problem_state.dF, J=dF_mat_state, P=None)
snes.setMonitor(lambda _, it, residual: print(it, residual))
snes.solve(None, vp)
problem_state.update_solutions(vp)  # TODO can this be safely removed?

In [ ]:
J_uncontrolled = MPI.sum(mesh.mpi_comm(), assemble_scalar(J))
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.1784542)

In [ ]:
plot(v, title="uncontrolled state velocity")

In [ ]:
plot(p, title="uncontrolled state pressure")

### Optimal control

In [ ]:
# Create problem associated to the optimality conditions
problem = NonlinearBlockProblem(F, dF, (v, p, u, z, b), bc, restriction)
F_vec = create_vector_block(F, restriction=restriction)
dF_mat = create_matrix_block(dF, restriction=(restriction, restriction))

In [ ]:
# Solve
vpuzb = create_vector_block(F, restriction=restriction)
snes = PETSc.SNES().create(mesh.mpi_comm())
snes.setTolerances(max_it=20)
snes.getKSP().setType("preonly")
snes.getKSP().getPC().setType("lu")
snes.getKSP().getPC().setFactorSolverType("mumps")
snes.setObjective(problem.obj)
snes.setFunction(problem.F, F_vec)
snes.setJacobian(problem.dF, J=dF_mat, P=None)
snes.setMonitor(lambda _, it, residual: print(it, residual))
snes.solve(None, vpuzb)
problem.update_solutions(vpuzb)  # TODO can this be safely removed?

In [ ]:
J_controlled = MPI.sum(mesh.mpi_comm(), assemble_scalar(J))
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 0.1249381)

In [ ]:
plot(v, title="state velocity")

In [ ]:
plot(p, title="state pressure")

In [ ]:
plot(u, title="control")

In [ ]:
plot(z, title="adjoint velocity")

In [ ]:
plot(b, title="adjoint pressure")